In [1]:
import pandas as pd
import spacy
from bertopic import BERTopic

import re
from datetime import datetime

from gensim.utils import simple_preprocess

import nltk
nltk.download(["stopwords", "rslp"])
stopwords = nltk.corpus.stopwords.words("portuguese")

import warnings
warnings.filterwarnings("ignore")

c:\Users\isabe\anaconda3\envs\tcc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\isabe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\isabe\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [2]:
df_reviews = pd.read_csv("guacamole_reviews.csv")
df_reviews.head()

,Date,Review
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já..."
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest..."
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó..."


In [3]:
df_comments = df_reviews["Review"]
df_dates = df_reviews["Date"]

In [4]:
def remove_stopwords(text):
    words = simple_preprocess(text)
    phrase_adjusted = " ".join([word for word in words if word not in stopwords])
    return phrase_adjusted.lower()

spacy_lemma = spacy.load("pt_core_news_sm")

def lemmatizer(text, postags_permitidos=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    doc = spacy_lemma(text.lower())
    doc_lemma = " ".join([token.lemma_ for token in doc if token.pos_ in postags_permitidos])
    return doc_lemma

In [5]:
def parse_date(date_str):
    months = {
        'janeiro': 'January', 'fevereiro': 'February', 'março': 'March', 'abril': 'April',
        'maio': 'May', 'junho': 'June', 'julho': 'July', 'agosto': 'August',
        'setembro': 'September', 'outubro': 'October', 'novembro': 'November', 'dezembro': 'December'
    }
    match = re.match(r'(\d+) de (\w+) de (\d+)', date_str)
    if match:
        day, month, year = match.groups()
        month = months[month.lower()]
        return datetime(int(year), list(months.values()).index(month) + 1, int(day)).strftime("%Y-%m-%d")
    else:
        return None

In [6]:
df_reviews["Review Lemma"] = df_reviews["Review"].map(remove_stopwords)
df_reviews["Review Lemma"] = df_reviews["Review"].map(lemmatizer)

In [7]:
df_reviews["Date Formatted"] = df_reviews["Date"].apply(parse_date)

In [8]:
df_reviews.head()

,Date,Review,Review Lemma,Date Formatted
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...,atendimento maravilhoso matheu muito atencioso...,2023-12-04
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...,gostar muito prato atendimento solicito comida...,2023-12-02
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já...",ameir bom cozinha mexicano já comi espaço muit...,2023-12-01
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest...",bom ambiente decoração funcionário prestativo ...,2023-11-29
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó...",amoma atendimento matheu bom amar atendimento ...,2023-11-27


In [9]:
df_comments = df_reviews["Review Lemma"]
doc_comments = df_comments.to_list()
doc_comments[:5]

['atendimento maravilhoso matheu muito atencioso colocar playlist atendente super simpatico',
 'gostar muito prato atendimento solicito comida saboroso certeza voltar mais vez',
 'ameir bom cozinha mexicano já comi espaço muito animar imersivo decoração casa linda atendimento excelente sempre',
 'bom ambiente decoração funcionário prestativo comida boa música agradável divertido mariachi',
 'amoma atendimento matheu bom amar atendimento sensacional comida muito saboroso']

In [10]:
topic_model = BERTopic(language="portuguese", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(doc_comments)

2023-12-10 11:21:58,987 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 90/90 [01:10<00:00,  1.28it/s]
2023-12-10 11:23:16,034 - BERTopic - Embedding - Completed ✓
2023-12-10 11:23:16,034 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-10 11:23:55,642 - BERTopic - Dimensionality - Completed ✓
2023-12-10 11:23:55,650 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-10 11:23:56,115 - BERTopic - Cluster - Completed ✓
2023-12-10 11:23:56,139 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-10 11:23:56,305 - BERTopic - Representation - Completed ✓


In [11]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,260,-1_muito_lugar_bom_mais,"[muito, lugar, bom, mais, aniversário, atendim...",[comida maravilhoso decoração incrível funcion...
1,0,735,0_mexicano_guacamole_restaurante_comida,"[mexicano, guacamole, restaurante, comida, mai...",[restaurante ter comida mexicano muito gostoso...
2,1,648,1_muito_bom_atendimento_ambiente,"[muito, bom, atendimento, ambiente, lugar, exc...",[experiência incrível comer ótimo lugar super ...
3,2,348,2_música_bom_ambiente_muito,"[música, bom, ambiente, muito, atendimento, mu...",[adorar local bom ir amigo decoração incrível ...
4,3,219,3_bebida_drink_cerveja_bom,"[bebida, drink, cerveja, bom, atendimento, gel...",[excelente muito bom atendimento lugar clima b...


In [12]:
topic_model.get_topic(0)

[('mexicano', 0.06841741161304897),
 ('guacamole', 0.03703690126466805),
 ('restaurante', 0.03260507770392968),
 ('comida', 0.032588234261462554),
 ('mais', 0.031381912519302516),
 ('bom', 0.029654041820110905),
 ('muito', 0.026828088558872365),
 ('ter', 0.024654843077058217),
 ('bem', 0.023371897233655452),
 ('ambiente', 0.02259199866474662)]

In [13]:
topic_model.visualize_barchart(top_n_topics=15)

In [14]:
topic_model.visualize_hierarchy()

In [15]:
dates_reviews = df_reviews["Date Formatted"].to_list()

In [16]:
topics_over_time = topic_model.topics_over_time(doc_comments, dates_reviews, datetime_format="%Y-%m-%d", nr_bins=20)

20it [00:01, 19.59it/s]


In [17]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)